# Hill Climb Racing Game Controller Using Hand Gestures

**Note**: This version has been modified to run without root privileges on Linux systems. The keyboard simulation functionality has been disabled, but the hand tracking and visual feedback still work. You'll see text indicating what actions would be taken (GAS/BRAKE/NEUTRAL), but it won't actually send keyboard commands to control the game.

To use the full functionality with keyboard control on Linux:
1. Run the notebook with root privileges (not recommended for security reasons)
2. Or use a different OS like Windows where the keyboard library doesn't require root access

# Importing Necessary Libraries

In [1]:
import cv2
import mediapipe
import traceback
import math
import keyboard

# Defining Functions

In [2]:
#It shows small points and coordinates on the tip of fingers

def show(i,frame):
    if i is not None:
        coor = mediapipe.solutions.drawing_utils._normalized_to_pixel_coordinates(i.x,i.y,fw,fh)
        if coor is not None:
            frame=cv2.circle(frame, coor, 5, (0, 0, 255), -1)
            cv2.putText(frame,str(coor),(coor[0]+10,coor[1]+10),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0, 0), 2)
    return frame

# Defining Keyboard Press & Release Function

In [3]:
#This function is used for displaying the control state without actually pressing keys
#Since the keyboard library requires root access on Linux

def keybd(ratio,frame):
    if ratio is not None:
        if ratio>=0.35:
            # Visual indication only - no actual keyboard press
            cv2.putText(frame,"GAS (would press right arrow)",(40,130),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255, 0), 2)
        elif ratio<0.2:
            # Visual indication only - no actual keyboard press  
            cv2.putText(frame,"BRAKE (would press left arrow)",(40,130),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0, 255), 2)
        else:
            # Visual indication only - no actual keyboard press
            cv2.putText(frame,"NEUTRAL",(40,130),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255, 255), 2)
    else:
        # Visual indication only - no actual keyboard press
        pass

# Main Code

In [4]:
# Instructions:- 
# NOTE: This is a demonstration version that doesn't actually control the game.
# It will show what actions would be taken but won't send keyboard commands.
# The original version requires running with root privileges on Linux.

try:
    
    
    #Getting Video Input from Camera
    capture = cv2.VideoCapture(0)
    fw = capture.get(cv2.CAP_PROP_FRAME_WIDTH)
    fh = capture.get(cv2.CAP_PROP_FRAME_HEIGHT)
    
    

# Detecting Fingers
    with mediapipe.solutions.hands.Hands(static_image_mode=False, min_detection_confidence=0.7, min_tracking_confidence=0.7, max_num_hands=1) as hands:
        while capture.isOpened():
            ret, frame = capture.read()
            if ret == False:
                continue
            
            frame = cv2.flip(frame, 1)
            results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            
            # Collecting coordinates of 6 hand landmarks
            if results.multi_hand_landmarks != None:
                thumb=results.multi_hand_landmarks[0].landmark[mediapipe.solutions.hands.HandLandmark.THUMB_TIP]
                index = results.multi_hand_landmarks[0].landmark[mediapipe.solutions.hands.HandLandmark.INDEX_FINGER_TIP]
                middle = results.multi_hand_landmarks[0].landmark[mediapipe.solutions.hands.HandLandmark.MIDDLE_FINGER_TIP]
                ring= results.multi_hand_landmarks[0].landmark[mediapipe.solutions.hands.HandLandmark.RING_FINGER_TIP]
                little=results.multi_hand_landmarks[0].landmark[mediapipe.solutions.hands.HandLandmark.PINKY_TIP]
                wri=results.multi_hand_landmarks[0].landmark[mediapipe.solutions.hands.HandLandmark.WRIST]

               
                a=[thumb,index,middle,ring,little]
                x0=0
                y0=0
                
                # Calculating the centroid of 5 finger's tips
                for i in a:
                    x0+=i.x
                    y0+=i.y
                c=(x0/5,y0/5)
                centroid = mediapipe.solutions.drawing_utils._normalized_to_pixel_coordinates(c[0],c[1],fw,fh)
                if centroid is not None:
                    frame=cv2.circle(frame, centroid, 5, (0, 255, 0), -1)
                    cv2.putText(frame,str(centroid),(centroid[0]+10,centroid[1]+10),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0, 0), 2)
                
                
                # Calculating the ratio(r/s) of distance between the centroid, avg of 5 fingers (r) and the centroid,wrist (s)
                wrist=mediapipe.solutions.drawing_utils._normalized_to_pixel_coordinates(wri.x,wri.y,fw,fh)
                if wrist is not None and centroid is not None and all(a):
                    frame=cv2.circle(frame, wrist, 5, (0, 0, 255), -1)
                    cv2.putText(frame,str(wrist),(wrist[0]+10,wrist[1]+10),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0, 0), 2)
                    dis=[]
                    for i in a:
                        v=mediapipe.solutions.drawing_utils._normalized_to_pixel_coordinates(i.x,i.y,fw,fh)
                        if v is not None:
                            dis.append(math.dist(v,centroid))
                    if len(dis)==5:
                        r=sum(dis)/5
                        s=math.dist(centroid,wrist)
                        cv2.putText(frame,"Ratio:- "+str(round(r/s,2)),(20, 70),cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0, 0), 2)
                                
                        #Showing control state without actual keyboard presses
                        keybd(r/s,frame)
                        
                frame=show(thumb,frame)
                frame=show(index,frame)
                frame=show(middle,frame)
                frame=show(ring,frame)
                frame=show(little,frame)
                
                
            cv2.imshow('HCR Game Controller (Demo Mode)', frame)
            
            if cv2.waitKey(1) == 27: #Escape(ESC.) key
                break
            
    cv2.destroyAllWindows()
    capture.release()

except:
    
    cv2.destroyAllWindows()
    capture.release()
    traceback.print_exc()

I0000 00:00:1759559472.617089  459503 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1759559472.648443  466982 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.3), renderer: Mesa Intel(R) HD Graphics 405 (BSW)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1759559472.935478  466971 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1759559473.086886  466972 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1759559502.309966  466972 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
Traceback (most recent call last):
  File "/tmp/ipykernel_459503/3117176172.py", line 65, in <mod

In [ ]:
# Alternative Solutions for Running with Full Keyboard Control

If you want the full functionality with keyboard control, you have these options:

## Option 1: Run with sudo (not recommended for security)
```bash
sudo jupyter notebook
```

## Option 2: Use PyAutoGUI instead of the keyboard library
PyAutoGUI is an alternative library that doesn't require root privileges on most systems.

```python
# Install PyAutoGUI
!pip install pyautogui

# Then replace keyboard functions with:
import pyautogui

# Instead of:
# keyboard.press("right arrow") 
# Use:
# pyautogui.keyDown("right")

# Instead of:
# keyboard.release("right arrow")
# Use: 
# pyautogui.keyUp("right")
```

## Option 3: Use on Windows
The keyboard library doesn't require admin privileges on Windows.

In [ ]:
# PyAutoGUI Implementation (uncomment and use this version if you want to try with PyAutoGUI)
# First install PyAutoGUI with: pip install pyautogui

"""
import pyautogui

def keybd_pyautogui(ratio, frame):
    if ratio is not None:
        if ratio >= 0.35:
            pyautogui.keyUp("left")
            pyautogui.keyDown("right")
            cv2.putText(frame, "GAS", (40, 130), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
        elif ratio < 0.2:
            pyautogui.keyUp("right")
            pyautogui.keyDown("left")
            cv2.putText(frame, "BRAKE", (40, 130), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
        else:
            pyautogui.keyUp("left")
            pyautogui.keyUp("right")
            cv2.putText(frame, "NEUTRAL", (40, 130), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 3)
    else:
        pyautogui.keyUp("left")
        pyautogui.keyUp("right")
"""